In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import cycler
from matplotlib.ticker import StrMethodFormatter
from matplotlib.axis import Axis
from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties
import sqlite3 as sq
from functools import reduce
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../..") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon

In [2]:
#functions
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

## Census Bureau

In [3]:
#2000
conn = sq.connect('../../../../GNRC-Data-Pipelines/Outputs/CensusBureau.db')
#SF12000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF1_2000]', conn)
sf12000 = pd.DataFrame(sql_query)
sf12000.drop(columns = 'Source', inplace = True)
#SF32000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF3_2000]', conn)
sf32000 = pd.DataFrame(sql_query)
sf32000.drop(columns = 'Source', inplace = True)
#TIGER2000
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2000]', conn)
TIGER2000 = pd.DataFrame(sql_query)
TIGER2000.drop(columns = 'Source', inplace = True)
dfs2000 = [TIGER2000, sf12000, sf32000]
twothousand= reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2000)
twothousand = twothousand.set_index(['NAME', 'GEO_ID'])
twothousandsuff = twothousand.add_suffix(' 2000')
#twothousand.columns = pd.MultiIndex.from_product([twothousand.columns, [2000], ['None']])
#2010
#SF12010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_SF1_2010]', conn)
sf12010 = pd.DataFrame(sql_query)
sf12010.drop(columns = 'Source', inplace = True)
#ACS52010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2010]', conn)
ACS52010 = pd.DataFrame(sql_query)
ACS52010.drop(columns = 'Source', inplace = True)
#TIGER2010
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2010]', conn)
TIGER2010 = pd.DataFrame(sql_query)
TIGER2010.drop(columns = 'Source', inplace = True)
dfs2010 = [TIGER2010, sf12010, ACS52010]
twothousandten= reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2010)
twothousandten = twothousandten.fillna(0)
twothousandten = twothousandten.set_index(['NAME', 'GEO_ID'])
twothousandtensuff = twothousandten.add_suffix(' 2010')
#twothousandten.columns = pd.MultiIndex.from_product([twothousandten.columns, [2010], ['None']])
#2020/21
#PL2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_PL_2020]', conn)
PL2020 = pd.DataFrame(sql_query)
PL2020.drop(columns = 'Source', inplace = True)
#ACS52021
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_ACS5_2021]', conn)
ACS52021 = pd.DataFrame(sql_query)
ACS52021.drop(columns = 'Source', inplace = True)
#TIGER2020
sql_query = pd.read_sql('SELECT * FROM [CensusBureau_TIGER_2020]', conn)
TIGER2020 = pd.DataFrame(sql_query)
TIGER2020.drop(columns = 'Source', inplace = True)
dfs2021 = [TIGER2020, ACS52021, PL2020]
twothousandtwentyone = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2021)
twothousandtwentyone = twothousandtwentyone.fillna(0)
twothousandtwentyone = twothousandtwentyone.set_index(['NAME', 'GEO_ID'])
#twothousandtwentyone = twothousandtwentyone.drop(columns = groupquarters)
twothousandtwentyonesuff = twothousandtwentyone.add_suffix(' Current')
#twothousandtwentyone.columns = pd.MultiIndex.from_product([twothousandtwentyone.columns, [2021], ['None']])

In [4]:
#CHECK WHATS GOING ON HERE LATER
twothousandten = twothousandten.drop(columns = 'index')

In [5]:
geos = ['Sumner County', 'Portland']

In [6]:
dfs = [twothousandsuff, twothousandtensuff, twothousandtwentyonesuff]
dfsuff = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME', 'GEO_ID'], how='outer'), dfs)

In [7]:
dfsuff = dfsuff.reset_index().set_index('NAME').transpose()
dfsuff = dfsuff.rename(columns = namestocommon)
dfsuff = dfsuff[geos]
dfsuff = dfsuff.transpose().reset_index()

In [8]:
cols = list(dfsuff.columns)
cols.remove('NAME')
cols.remove('GEO_ID')
dfsuff[cols] = dfsuff[cols].astype(float)

In [9]:
dfsuff.head()

,NAME,GEO_ID,Square Acres 2000,Square Miles 2000,Population 2000,Male Under 5 2000,Female Under 5 2000,Male 5 to 14 2000,Female 5 to 14 2000,Male 15 to 24 2000,Female 15 to 24 2000,Male 5 to 9 2000,Female 5 to 9 2000,Male 10 to 14 2000,Female 10 to 14 2000,Male 15 to 17 2000,Female 15 to 17 2000,Male 18 to 24 2000,Female 18 to 24 2000,Male 25 to 34 2000,Female 25 to 34 2000,Male 35 to 44 2000,Female 35 to 44 2000,Male 45 to 54 2000,Female 45 to 54 2000,Male 55 to 64 2000,Female 55 to 64 2000,Male 65 to 74 2000,Female 65 to 74 2000,Male 75 to 84 2000,Female 75 to 84 2000,Male 85 and Older 2000,Female 85 and Older 2000,Age:Under 5 2000,Age %:Under 5 2000,Age:5 to 9 2000,Age%:5 to 9 2000,Age:10 to 14 2000,Age%:10 to 14 2000,Age:15 to 17 2000,Age%:15 to 17 2000,Age:18 to 24 2000,Age%:18 to 24 2000,Age:25 to 34 2000,Age%:25 to 34 2000,Age:35 to 44 2000,Age%:35 to 44 2000,Age:45 to 54 2000,Age%:45 to 54 2000,Age:55 to 64 2000,Age%:55 to 64 2000,Age:65 to 74 2000,Age%:65 to 74 2000,Age:75 to 84 2000,Age%:75 to 84 2000,Age:85 and Older 2000,Age%:85 and Older 2000,Age:School Age 5 to 18 2000,Age%:School Age 5 to 18 2000,Age:Under 18 2000,Age%:Under 18 2000,Age:18 to 54 2000,Age%:18 to 54 2000,Age:18 to 64 2000,Age%:18 to 64 2000,Age:55 and Older 2000,Age%:55 and Older 2000,Age:65 and Older 2000,Age%:65 and Older 2000,White Alone 2000,White Alone % 2000,Non-White Population: Race 2000,Black or African American Alone 2000,Black or African American Alone % 2000,Black or African American Alone % Non-White Population 2000,American Indian Alaska Native Alone 2000,American Indian Alaska Native Alone % 2000,American Indian Alaska Native Alone % Non-White Population 2000,Asian Alone 2000,Asian Alone % 2000,Asian Alone % Non-White Population 2000,Native Hawaiian Other Pacific Islander Alone 2000,Native Hawaiian Other Pacific Islander Alone % 2000,Native Hawaiian Other Pacific Islander Alone % Non-White Population 2000,Some Other Race Alone 2000,Some Other Race Alone % 2000,Some Other Race Alone % Non-White Population 2000,Two or More Races 2000,Two or More Races % 2000,Two or More Races % Non-White Population 2000,Hispanic or Latino 2000,Hispanic or Latino % 2000,Not Hispanic or Latino 2000,Not Hispanic or Latino % 2000,Minority 2000,Minority % 2000,White Alone Not Hispanic or Latino 2000,White Alone Not Hispanic or Latino % 2000,Total Households 2000,Family Households 2000,Family Households: Married Couple Family 2000,"Household Type%: Family, Married Couple Households 2000",Family Households: Not Married Couple Family 2000,"Household Type%: Family, Not Married Couple Households 2000",Family Households: Not Married Couple: Male no Spouse 2000,Family Households: Not Married Couple: Female no Spouse 2000,Nonfamily Households 2000,Nonfamily Households: Householder Alone 2000,"Household Type%: Nonfamily, Householder Alone 2000",Nonfamily Households: Householder not Alone 2000,"Household Type%: Nonfamily, Householder Not Alone 2000",Average Household Size 2000,Occupancy:Total Households 2000,Occupancy:Occupied Units 2000,Occupancy%:Occupied Units 2000,Occupancy:Vacant Units 2000,Occupancy%:Vacant Units 2000,Tenure:Total Households 2000,Tenure:Owners 2000,Tenure%:Owners 2000,Tenure:Renters 2000,Tenure%:Renters 2000,Median Household Income 2000,HHIncome:Total Households 2000,"HHIncome:Less than 10,000 2000","HHIncome%:Less than 10,000 2000","HHIncome:10 to 14,999 2000","HHIncome%:10 to 14,999 2000","HHIncome:15 to 19,999 2000","HHIncome%:15 to 19,999 2000","HHIncome:20 to 24,999 2000","HHIncome%:20 to 24,999 2000","HHIncome:25 to 29,999 2000","HHIncome%:25 to 29,999 2000","HHIncome:30 to 34,999 2000","HHIncome%:30 to 34,999 2000","HHIncome:35 to 39,999 2000","HHIncome%:35 to 39,999 2000","HHIncome:40 to 44,999 2000","HHIncome%:40 to 44,999 2000","HHIncome:45 to 49,999 2000","HHIncome%:45 to 49,999 2000","HHIncome:50 to 59,999 2000","HHIncome%:50 to 59,999 2000","HHIncome:60 to 74,999 2000","HHIncome%:60 to 74,999 2000","HHIncome:75 to 99,999 2000","HH

In [10]:
cols = ['NAME', 'Population 2010', 'Population Current', 'Minority Current', 'Minority % Current', 'Age:School Age 5 to 18 Current', 'Age%:School Age 5 to 18 Current', 
        'Poverty: Population Below Poverty Level Current', 'Poverty%: Below Poverty Level Current', 'Age:55 and Older Current', 'Age%:55 and Older Current', 
        'Age:65 and Older Current', 'Age%:65 and Older Current', 'Units:Total Series 2010', 'Units:Total Series Current', 
        'Units:1 Unit, Detached Current', 'Units:1 Unit, Attached Current', 'Units:Mobile Home Current',
        'Units%:1 Unit, Detached Current', 'Units%:1 Unit, Attached Current', 'Units%:Mobile Home Current', 'Median Household Income Current', 
        'CB:Cost Burdened Homeowners Current', 'CB:Cost Burdened Renters Current',
        'CB%: Cost Burdened Homeowners Current', 'CB%:Cost Burdened Renters Current', 'Average Household Size Current', 'Tenure:Owners Current', 'Tenure%:Owners Current', 
        'Tenure:Renters Current', 'Tenure%:Renters Current', 'Square Acres Current']

In [11]:
acs = dfsuff[cols]

In [12]:
acs['Population Increase 2010 to 2020'] = acs['Population Current'] - acs['Population 2010']
months = 12 * 10
acs['Population Gained Per Month 2010 to 2020'] = acs['Population Increase 2010 to 2020']/months
acs['Housing Unit Increase 2010 to 2020'] = acs['Units:Total Series Current'] - acs['Units:Total Series 2010']
acs['Housing Units Gained Per Month 2010 to 2020'] = acs['Housing Unit Increase 2010 to 2020']/months

In [13]:
thelist = [acs['Units:1 Unit, Detached Current'], acs['Units:1 Unit, Attached Current'], acs['Units:Mobile Home Current']]
acs['Units:All Multi-Family Current'] = acs['Units:Total Series Current'] - sum(thelist)
acs['Units%:All Multi-Family Current'] = percent(acs['Units:All Multi-Family Current'], acs['Units:Total Series Current'])

In [14]:
acs = acs.set_index('NAME').transpose()
acs

NAME,Sumner County,Portland
Population 2010,160645.000000,11480.000000
Population Current,196281.000000,13156.000000
Minority Current,41112.000000,2329.000000
Minority % Current,20.945481,17.702949
Age:School Age 5 to 18 Current,34161.000000,2674.000000
Age%:School Age 5 to 18 Current,17.696699,20.520298
Poverty: Population Below Poverty Level Current,18752.000000,1851.000000
Poverty%: Below Poverty Level Current,9.820938,14.323300
Age:55 and Older Current,56226.000000,2674.000000
Age%:55 and Older Current,29.127209,20.520298


## JobsEQ

In [30]:
conn = sq.connect('../../../../GNRC-Data-Pipelines/Outputs/Labor_Economy.db')
sql_query = pd.read_sql('SELECT * FROM [JobsEQ_IndustryEmployment_Annual_Change]', conn)
data = pd.DataFrame(sql_query)
data = data.loc[data['NAME'] == 'Portland city, Tennessee']
data = data.loc[(data['Year'] == '2010')|(data['Year'] == '2020')|(data['Time Frame'] == '2010-2020')].reset_index(drop = True)
data = data.drop(columns = ['GEO_ID', 'Source'])

In [31]:
cols = ['NAME', 'Year', 'Time Frame', 'Ind:Total', 'Ind:Total Change', 'Ind:Total % Change', 'Ind%:Snapshot: Blue Collar', 'Ind%:Snapshot: White Collar', 
        'Ind%:Snapshot: Government', 
        'Ind%:Snapshot: Industrial, Manufacturing, Utilities, & Logistics', 
        'Ind%:Snapshot: Professional Services', 'Ind%:Snapshot: Retail & Hospitality', 
        'Ind:Snapshot: Blue Collar', 'Ind:Snapshot: White Collar', 'Ind:Snapshot: Government', 'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics', 
        'Ind:Snapshot: Professional Services', 'Ind:Snapshot: Retail & Hospitality', 
        'Ind:Snapshot: Blue Collar Change', 'Ind:Snapshot: White Collar Change', 'Ind:Snapshot: Government Change', 
        'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics Change', 
        'Ind:Snapshot: Professional Services Change', 'Ind:Snapshot: Retail & Hospitality Change',
        'Ind:Snapshot: Blue Collar % Change', 'Ind:Snapshot: White Collar % Change', 'Ind:Snapshot: Government % Change', 
        'Ind:Snapshot: Industrial, Manufacturing, Utilities, & Logistics % Change', 
        'Ind:Snapshot: Professional Services % Change', 'Ind:Snapshot: Retail & Hospitality % Change']
data = data[cols]

In [32]:
data = data.set_index(['NAME', 'Year', 'Time Frame']).transpose()
data

NAME                                               Portland city, Tennessee                          
Year                                                                   2010         2020         None
Time Frame                                                             None         None    2010-2020
Ind:Total                                                       5141.148598  7408.726855          NaN
Ind:Total Change                                                        NaN          NaN  2267.578256
Ind:Total % Change                                                      NaN          NaN    44.106452
Ind%:Snapshot: Blue Collar                                        58.408262    65.170321          NaN
Ind%:Snapshot: White Collar                                       21.374641    15.227690          NaN
Ind%:Snapshot: Government                                          2.847940     3.029178          NaN
Ind%:Snapshot: Industrial, Manufacturing, Utili...                58.700680    65.328531          NaN
Ind%:Snapshot: Professional Services                              27.267846    20.821062          NaN
Ind%:Snapshot: Retail & Hospitality                               11.183534    10.821229          NaN
Ind:Snapshot: Blue Collar                                       3002.855552  4828.291076          NaN
Ind:Snapshot: White Collar                                      1098.902077  1128.177979          NaN
Ind:Snapshot: Government                                         146.416852   224.423501          NaN
Ind:Snapshot: Industrial, Manufacturing, Utilit...              3017.889171  4840.012431          NaN
Ind:Snapshot: Professional Services                             1401.880485  1542.575590          NaN
Ind:Snapshot: Retail & Hospitality                               574.962090   801.715333          NaN
Ind:Snapshot: Blue Collar Change                                        NaN          NaN  1825.435524
Ind:Snapshot: White Collar Change                                       NaN          NaN    29.275902
Ind:Snapshot: Government Change                                         NaN          NaN    78.006649
Ind:Snapshot: Industrial, Manufacturing, Utilit...                      NaN          NaN  1822.123260
Ind:Snapshot: Professional Services Change                              NaN          NaN   140.695105
Ind:Snapshot: Retail & Hospitality Change                               NaN          NaN   226.753242
Ind:Snapshot: Blue Collar % Change                                      NaN          NaN    60.789988
Ind:Snapshot: White Collar % Change                                     NaN          NaN     2.664105
Ind:Snapshot: Government % Change                                       NaN          NaN    53.277097
Ind:Snapshot: Industrial, Manufacturing, Utilit...                      NaN          NaN    60.377408
Ind:Snapshot: Professional Services % Change                            NaN          NaN    10.036170
Ind:Snapshot: Retail & Hospitality % Change                             NaN          NaN    39.437947